In [1]:
from analytics.env import *

$\int_0^\infty x dx$

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.base import TransformerMixin
from sklearn import cross_validation, metrics

from sklearn.manifold import TSNE, TSNE

In [3]:
from sklearn import linear_model
from sklearn.neural_network import BernoulliRBM
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
import  sklearn.neural_network as nn
import sklearn.decomposition as dec
from sklearn import svm
import sklearn.ensemble as ens

In [4]:
from internaldata import Database, Classifier

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
clf = Classifier()

In [6]:
db=Database()
data = db.load_data()

In [7]:
data.head()

,Datum,Deleted,Hash,Kategorie,Lastschrift,Text,Unnamed: 6,Unnamed: 7
0,2014-01-03,False,6848fb24b03ed71e56d98fd8f3af3401,Anschaffungen,35.00,Zürcher Kantonalbank CH2300700350040352767 K...,NaN,NaN
1,2014-01-03,True,868f4d0ba29c129f408f979f3a9f8e0e,NaN,200.00,BARGELDBEZUG\nVOM 01.01.2014\nKARTEN NR. 64186...,NaN,NaN
2,2014-01-03,False,3c3550b40b42a3367a2cb4b29d28412d,Krankenkasse,176.75,01-37897-1 100000013241880130751593563\nAssur...,NaN,NaN
3,2014-01-03,False,5c42bb51a03c0c035f074a9a9b8ddcbb,Ausgang,58.00,KAUF/DIENSTLEISTUNG\nVOM 31.12.2013\nKARTEN NR...,NaN,NaN
4,2014-01-05,False,cf97222b176cb612026a061b0757d230,Sparen,-1000.00,ÜBERTRAG\nAUS KONTO 92-900275-2\nSCHOCH TOBIAS...,NaN,NaN


In [8]:
class DenseTransformer(TransformerMixin):

        def transform(self, X, y=None, **fit_params):
            return X.todense()

        def fit_transform(self, X, y=None, **fit_params):
            self.fit(X, y, **fit_params)
            return self.transform(X)

        def fit(self, X, y=None, **fit_params):
            return self

In [26]:
categories = np.unique(data.Kategorie).tolist()
target = np.array([categories.index(cat) for cat in data.Kategorie.tolist()])

In [27]:
categories

[nan,
 'Anschaffungen',
 'Ausbildung',
 'Ausgang',
 'Bekleidung',
 'Bussen',
 'B\xc3\xbccher',
 'Eishockey',
 'Essen',
 'Geb\xc3\xbchren',
 'Geschenke',
 'Krankenkasse',
 'Lohn',
 'Medizin',
 'Miete',
 'Mobiltelefon',
 'Reisen/Ausfl\xc3\xbcge',
 'Schulden',
 'Sparen',
 'Spenden',
 'Sport allgemein',
 'Steuern',
 'Transport',
 'Vorsorge',
 'Wellness/Coiffeur']

In [40]:
data.Kategorie[data.Deleted]=np.nan

In [41]:
data_train, data_test, target_train, target_test = cross_validation.train_test_split(data.Text, target, test_size=0.1)

In [29]:
features = Pipeline([('vect',CountVectorizer(ngram_range=(2,5),analyzer='char_wb',lowercase=True)),
                     ('trans',TfidfTransformer(use_idf=True))])

In [30]:
counts = features.fit_transform(data.Text)
counts.min()

In [8]:
tsne = TSNE(init='pca',verbose=True,perplexity=30,n_iter=2000,n_iter_without_progress=50,learning_rate=1500,
            early_exaggeration=5)
f_red = tsne.fit_transform(features.fit_transform(data.Text).toarray())

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 1437
[t-SNE] Computed conditional probabilities for sample 1437 / 1437
[t-SNE] Mean sigma: 0.412276
[t-SNE] KL divergence after 100 iterations with early exaggeration: 0.719831
[t-SNE] Error after 175 iterations: 0.719831


In [10]:
pd.DataFrame(f_red,columns=['X','Y']).plot(kind='scatter',x='X',y='Y',c=data.Kategorie.cat.codes,colormap='viridis');

In [15]:
features.fit_transform(data.Text)

<1437x27770 sparse matrix of type '<type 'numpy.float64'>'
	with 282174 stored elements in Compressed Sparse Row format>

In [42]:
text_clf = Pipeline([('vect',CountVectorizer()),
                     ('tfidf',TfidfTransformer()),
                     # ('dense',DenseTransformer()),
                     # ('tsne',tsne)
                     # ('logistic', linear_model.LogisticRegression())])
                     #('clf',ens.forest.RandomForestClassifier())])
                     # ('clf',svm.SVC())])
                     ('clf', SGDClassifier(loss='modified_huber', penalty='l2',random_state=42))])

In [43]:
parameters = {'vect__ngram_range': [(1,4),(1,5),(2,4),(2,5)],
              'vect__analyzer':['char_wb'],
              'vect__lowercase': [True],
              'tfidf__use_idf': [True],
              #'clf__max_depth': [None,10],
              #'clf__min_samples_split': [2,5]
              # 'rbm__learning_rate': [0.05,0.1],
              # 'rbm__n_iter': [25],
              # 'rbm__n_components': [50,100,150],
              #'logistic__fit_intercept':[True,False],
              # 'logistic__C': [1000]
              # }
              #'clf__kernel': ['rbf'],
              #'clf__C':np.linspace(0.02,0.6,10),'clf__gamma':np.linspace(0.02,0.05,3),
              'clf__alpha': np.linspace(1e-2,1e-5,10)
             }

In [44]:
gs_clf = GridSearchCV(text_clf,parameters,cv=5,verbose=1)
gs_clf.fit(data_train,target_train)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   17.2s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': [(1, 4), (1, 5), (2, 4), (2, 5)], 'vect__analyzer': ['char_wb'], 'vect__lowercase': [True], 'tfidf__use_idf': [True], 'clf__alpha': array([  1.00000e-02,   8.89000e-03,   7.78000e-03,   6.67000e-03,
         5.56000e-03,   4.45000e-03,   3.34000e-03,   2.23000e-03,
         1.12000e-03,   1.00000e-05])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [45]:
print gs_clf.best_estimator_

Pipeline(steps=[('vect', CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 5), preprocessor=None, stop_words=None,
        ...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])


In [46]:
preds = gs_clf.predict(data_test)

In [49]:
preds_all = gs_clf.predict(data.Text)

In [51]:
print metrics.classification_report(target,preds_all)

             precision    recall  f1-score   support

          0       0.98      0.98      0.98       148
          1       1.00      0.85      0.92        33
          2       1.00      1.00      1.00        15
          3       0.95      0.97      0.96       182
          4       0.96      1.00      0.98        49
          5       1.00      0.75      0.86         4
          6       1.00      1.00      1.00        11
          7       0.95      0.98      0.97        61
          8       0.95      0.98      0.97       322
          9       0.97      0.97      0.97        31
         10       0.98      0.89      0.93        53
         11       1.00      1.00      1.00        38
         12       1.00      0.98      0.99        47
         13       1.00      0.88      0.94        17
         14       0.93      0.95      0.94        65
         15       0.98      1.00      0.99        40
         16       0.96      0.95      0.96       189
         17       1.00      1.00      1.00   

In [47]:
print metrics.classification_report(target_test, preds)

             precision    recall  f1-score   support

          0       0.94      0.94      0.94        17
          1       1.00      0.25      0.40         4
          3       0.82      0.78      0.80        18
          4       0.83      1.00      0.91         5
          5       0.00      0.00      0.00         1
          6       1.00      1.00      1.00         1
          7       0.86      1.00      0.92         6
          8       0.84      1.00      0.91        31
          9       0.50      0.50      0.50         2
         10       0.75      0.75      0.75         4
         11       1.00      1.00      1.00         5
         12       1.00      1.00      1.00         4
         13       1.00      0.67      0.80         3
         14       1.00      0.75      0.86         4
         15       1.00      1.00      1.00         3
         16       0.80      0.84      0.82        19
         17       1.00      1.00      1.00         3
         18       1.00      1.00      1.00   

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
print metrics.confusion_matrix(target_test,preds)

[[16  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  2  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 31  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0

In [53]:
text_clf = gs_clf.best_estimator_
text_clf.classes_names = map(str,categories)

text_clf.TRAINING_DATE = datetime.datetime.now()
text_clf.TRAINING_SAMPLES = data_train.shape[0]
text_clf.TEST_SAMPLES = data_test.shape[0]
text_clf.SCORE = metrics.precision_recall_fscore_support(target_test,preds)

In [54]:
import pickle
with open(clf.properties,"wb+") as fp:
    pickle.dump(text_clf,fp)

In [13]:
import h2o

In [14]:
h2o.init()

H2O cluster uptime:,52 seconds 487 milliseconds
H2O cluster version:,3.8.2.9
H2O cluster name:,H2O_started_from_python_tobi_ftl137
H2O cluster total nodes:,1
H2O cluster total free memory:,3.14 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [109]:
train = pd.DataFrame({'data':data_train.values,'target':target_train})

In [111]:
hdf_train = h2o.H2OFrame({'A':data_train.str.replace('\n','').values.tolist(),'B':target_train.tolist()})
hdf_train


Parse Progress: [##################################################] 100%


A,B
GIRO AUS KONTO 87-223037-4ALOS COLOMER NURIADIETZINGERSTRASSE 68003 ZÜRICHMITTEILUNGEN:ALLESGUTICARI! -GEBURSTAGSGESCHENK,7
01-37897-1 100000013241880140812225060Assura - Assurance Assura / Assura SAmaladie et accidents1009 Pully,11
Pasta Station vor Match,8
Coop-4899 Zürich Bahnh Zürich 1Lebensmittelgeschäft,8
KAUF/DIENSTLEISTUNGVOM 25.09.2014KARTEN NR. XXXX6358MAISON TRUFFE AGSTÄFA,8
AUTOMATISCHE ABDISPOSITIONAUF KONTO 92-781711-8SCHOCH TOBIASZÜRICH,18
Gipfeli,8
Cappuccino & Gipfeli Bahnhof,8
THE BOX NEW YORKRestaurant,16
BARGELDBEZUG VOM 26.03.2014KARTEN NR. 64186358POSTOMAT: BAHNHOF SBBZÜRICH,0


In [112]:
hdf_test = h2o.H2OFrame({'A':data_test.str.replace('\n','').values.tolist(),'B':target_test.tolist()})
hdf_test


Parse Progress: [##################################################] 100%


A,B
GIRO AUS KONTO 87-223037-4ALOS COLOMER NURIADIETZINGERSTRASSE 68003 ZÜRICHMITTEILUNGEN:YEN'S + VARIOS,3
LATINOS DINER GIBRALTARRestaurant,16
Bratwurst nach Match in Reinach,8
E-FINANCE 87-71996-7RAIFFEISEN SCHWEIZCH1081487000040532344TOBIAS SCHOCHDIETZINGERSTRASSE 6,14
KAUF/DIENSTLEISTUNGVOM 24.09.2014KARTEN NR. XXXX6358MAISON TRUFFE AGSTÄFA,8
Asia Shop Curry,8
01-1093-0 130994000128Stadtpolizei Parkbusse Sensirion ClaraZürich,5
Geburtstagsfest Drink zusätzlich,3
BARGELDBEZUG VOM 31.03.2014KARTEN NR. XXXX6358POSTOMAT: POSTSTELLE SIHLPOSTZÜRICH,0
Schlittschuhe schleifen,8


In [61]:
import h2o.demo

In [113]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [83]:
anomaly_model = H2OAutoEncoderEstimator( 
        activation="Tanh", 
        hidden=[50,50,50], 
        sparse=True,
        l1=1e-4, 
        epochs=100)               

In [84]:
anomaly_model.train(x='A',training_frame=hdf_train) 


deeplearning Model Build Progress: [##################################################] 100%


In [88]:
recon_error = anomaly_model.anomaly(hdf_test)


# Note: Testing = Reconstructing the test dataset
test_recon = anomaly_model.predict(hdf_test) 


deeplearning prediction Progress: [##################################################] 100%


reconstr_A.01-1093-0 130994000128Stadtpolizei Parkbusse Sensirion ClaraZürich reconstr_A.01-1219-9 144844315400099990108874878medica Blutuntersuchung Check-upMEDIZINISCHELABORATORIENDr. F. KAEPPELI AGZürich reconstr_A.01-12807-2 900000000003160570000015008Möbel Pfister AG Weihnachtsgeschenk Priska&Urban5034 Suhr reconstr_A.01-14086-6 1403242822199169BonusCard.ch AGVisa Card MärzZürich reconstr_A.01-14086-6 1405251822199169BonusCard.ch AGVisa Card Mai 2014Zürich reconstr_A.01-14086-6 1409231822199166BonusCard.ch AGVISA Kreditkarte September 2014Zürich reconstr_A.01-14086-6 1406231822199161BonusCard.ch AG Visa Card Juni 2014Zürich reconstr_A.01-14086-6 1408241822199169BonusCard.ch AG Visa Card August 2014Zürich reconstr_A.01-14086-6 1411241822199160BonusCard.ch AG Visa Karte NovemberZürich reconstr_A.01-14086-6 1412231822199161BonusCard.ch AG VISA Kreditkarte DezemberZürich reconstr_A.01-14086-6 1501261822199161BonusCard.ch AG Visa Card Januar 2015Zürich reconstr_A.01-145-6263213000000002015000371337UBS AG Zoll und Einführgebühren Headphones8098 Zürich reconstr_A.01-1518-7 810309000000017372140284674Migros Bank AGZürich reconstr_A.01-1518-7 810309000000017372140292370Migros Bank AGZahnarzt FüllungserneuerungZürich reconstr_A.01-15626-0 214735057049530ELV Elektronik AGLED Lampen WohnzimmerPostfach 1000, 4452 ItingenDE-Leer reconstr_A.01-1852-7 322192000000000001000693203Thurgauer Kantonalbank Beleg Nr. 455428Weinfelden reconstr_A.01-200027-2 110018464790000000000004462Zürcher Kantonalbank Skateathon EHCW2Zürich reconstr_A.01-200027-2 929484500199874620120000008Zürcher KantonalbankZürich reconstr_A.01-200027-2 929484500199874620140000000Zürcher Kantonalbank Bundessteuern 2014Zürich reconstr_A.01-200027-2 943504000000002075006105151Zürcher Kantonalbank Mitgliederbeitrag EHCW2Zürich reconstr_A.01-200027-2110018464790000000000005187Zürcher KantonalbankREF: 518 reconstr_A.01-200027-2929484500199874620130000003Zürcher KantonalbankBundessteuern 2013Zürich reconstr_A.01-200027-2929484500199874620150000004Zürcher KantonalbankBundessteuern 2015Zürich reconstr_A.01-200027-2933411000000019803851160019Zürcher KantonalbankZahnröntgenaufnahme Uni ZürichZürich reconstr_A.01-200027-2933411000000019803851160027Zürcher KantonalbankRechnung Nr. 453845Zürich reconstr_A.01-200027-2933411000000019803851160035Zürcher KantonalbankRechnung Nr. 454353Zürich reconstr_A.01-200027-2943504000000002075006097278Zürcher KantonalbankMitgliederbeitrag 14/15Zürich reconstr_A.01-2462-6 9954140130615413Stadtpolizei WinterthurGeschwindigkeitsbusseOrdnungsbussenzentrale8402 Winterthur reconstr_A.01-2622-6 961307001000023909559849564Credit Suisse Rasperry Pi Zubehör8603 Schwerzenbach reconstr_A.01-2622-6 961307001000023909559884203Credit Suisse Rasperry Pi8603 Schwerzenbach reconstr_A.01-2622-6 961307001000023909560064238Credit Suisse Rasperry Pi Zubehör 28603 Schwerzenbach reconstr_A.01-2622-6 961307001000023909560134922Credit Suisse Rasperry Pi Zubehör 38603 Schwerzenbach reconstr_A.01-2654-0942971000000001312300681755Credit Suisse8070 Zürich reconstr_A.01-2654-0942971000000001408250799502Credit Suisse Arztbesuch Muttermal8070 Zürich reconstr_A.01-32555-6 200061599984909001929558896Helsana Versicherungen AG Helsana Versicherungen AGPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32555-6 200061599984909002132926996Helsana Versicherungen AG Helsana Versicherungen AGPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32555-6 200061599984909001969103008Helsana Versicherungen AG Helsana Versicherungen AGPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32555-6 200061599984909002010796839Helsana Versicherungen AG Helsana Versicherungen AGPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32555-6 200061599984909002048406210Helsana Versicherungen AG Helsana Insurance LtdPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32555-6 200061599984909002092000784Helsana Versicherungen AG Helsana Versicherungen AGPraemien-Inkasso EV8600 Dübendorf reconstr_A.01-32595-8 3074659468104214Orell Füssli 

In [117]:
model = H2ODeepLearningEstimator(
distribution="multinomial",
activation="RectifierWithDropout",
hidden=[32,32,32],
input_dropout_ratio=0.2,
sparse=True,
l1=1e-5,
epochs=1000,nfolds=10)

In [118]:
hdf_train['B'] = hdf_train['B'].asfactor()

In [119]:
hdf_test['A']=hdf_test['A'].asfactor()

In [120]:
model.train(
x='A',
y='B',
training_frame=hdf_train,
validation_frame=hdf_test)


deeplearning Model Build Progress: [##################################################] 100%


In [103]:
model.predict(hdf_test)


deeplearning prediction Progress: [##################################################] 100%


predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995
8,0.0941644,0.0220718,0.00868842,0.12046,0.0307304,0.00417091,0.00291895,0.0396251,0.214695,0.0202652,0.0340906,0.0249917,0.0307136,0.00958952,0.0480339,0.0217579,0.131112,0.0192294,0.0206002,0.00662254,0.0258384,0.00618678,0.0408611,0.0192926,0.00328995


In [121]:
model.model_performance()


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.827739313484
R^2: 0.980577606996
LogLoss: 2.97251243662
Mean Per-Class Error: 0.96

Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Error,Rate
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,137 / 137
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,27 / 27
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14 / 14
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,150 / 150
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45 / 45
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12 / 12
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,61 / 61
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26 / 26
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4 / 4



See the whole table with table.as_data_frame()

Top-10 Hit Ratios: 


k,hit_ratio
1,0.1226551
2,0.2308802
3,0.4458875
4,0.5447331
5,0.5721500
6,0.6038961
7,0.6464646
8,0.6904762
9,0.7113997
10,0.7366522
